In [1]:
pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python==4.6.0.66

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cmake


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install dlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import hypot
import pyautogui
import dlib

Step 1: Real-time Face Detection

Step 2: Find the landmarks for the detected face

Step 3: Build the Jump Control mechanism for the Dino

Step 4: Build the Crouch Control mechanism

Step 5: Perform Calibration

Step 6: Keyboard Automation with PyautoGUI

Step 7: Build the Final Application

## 1: Face Detection

We can use OpenCV’s Haar cascades or Dlib’s HOG-based face detector but instead, we are going to use a more robust deep learning-based SSD face detector with OpenCV’s DNN module.

Initialize The DNN Module:

The SSD face detector provided by OpenCV is a Caffe model and you will need two files to do inference with it. A .prototxt file that defines the model architecture and a .caffemodel file that contains the pre-trained weights for the layers in the architecture.

In [6]:
import os

In [7]:
import os

for path in ['deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000_fp16.caffemodel']:
    print(os.path.exists(path))

False
False


In [8]:
# Required file path
model_weights =  "dependencies/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# architecture file path
model_arch = "dependencies/deploy.prototxt.txt"

# Load the caffe model
net = cv2.dnn.readNetFromCaffe(model_arch, model_weights)
#net = cv2.dnn.readNetFromCaffe( model_weights,model_arch)

In [9]:
def face_detector(image, threshold =0.7):   # In thresholding, we convert an image from colour or grayscale into a binary image
    
    # Get the height,width of the image
    h, w = image.shape[:2]

    # Apply mean subtraction, and create 4D blob from image
    blob = cv2.dnn.blobFromImage(image, 1.0,(300, 300), (104.0, 117.0, 123.0))
    
    # Set the new input value for the network
    net.setInput(blob)
    
    # Run forward pass on the input to compute output
    faces = net.forward()
    
    # Get the confidence value for all detected faces
    prediction_scores = faces[:,:,:,2]
    
    # Get the index of the prediction with highest confidence 
    i = np.argmax(prediction_scores)
    
    # Get the face with highest confidence 
    face = faces[0,0,i]
    
    # Extract the confidence
    confidence = face[2]
    
    # if confidence value is greater than the threshold
    if confidence > threshold:   # will conside face with highest confidence as we only want single face
        
        # The 4 values at indexes 3 to 6 are the top-left, bottom-right coordinates
        # scales to range 0-1.The original coordinates can be found by 
        # multiplying x,y values with the width,height of the image
        box = face[3:7] * np.array([w, h, w, h])
        
        # The coordinates are the pixel numbers relative to the top left
        # corner of the image therefore needs be quantized to int type
        (x1, y1, x2, y2) = box.astype("int")
        
        # Draw a bounding box around the face.
        annotated_frame = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0, 0, 255), 2)
        output = (annotated_frame, (x1, y1, x2, y2), True, confidence)
    
    # Return the original frame if no face is detected with high confidence.
    else:
        output = (image,(),False, 0)
    
    
    return output

In [10]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)   # Create a video capture object, in this case we are reading the video from a file


# Set the window to a normal one so we can adjust it
cv2.namedWindow('Face Detection', cv2.WINDOW_NORMAL) 

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    annotated_frame, coords, status, conf = face_detector(frame)
    
    # Display the frame
    cv2.imshow('Face Detection', annotated_frame)
    
    # Break the loop if ESC key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

## Step 2: Landmarks Detection


To implement the jump mechanism, we need information about whether the mouth is open or closed. For this, we need to detect facial landmarks so we can determine the position of upper and lower lips.

By using Dlib’s 68 landmark detector, we’ll be able to detect below 68 landmarks on the face.



Download model from here - https://github.com/italojs/facial-landmarks-recognition/blob/master/shape_predictor_68_face_landmarks.dat

To initialize the landmark detector, you will use dlib.shape_predictor() function, which will load the pre-trained landmark detector from the disk.

In [ ]:
# initialize the landmark detector

predictor = dlib.shape_predictor("dependencies/shape_predictor_68_face_landmarks.dat")

#### Create the detect_landmarks() function


Now we will create a function called detect_landmarks() which takes in the coordinates of the detected face returned by the face_detector() method and then detects landmarks and returns them, while also annotating the image with circles on landmark positions.

In [ ]:
def detect_landmarks(box, image):
    
    # For faster results convert the image to gray-scale
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the coordinates 
    (x1, y1, x2, y2) = box

    # Perform the detection
    shape = predictor(gray_scale, dlib.rectangle(x1, y1, x2, y2))
    
    # Get the numPy array containing the coordinates of the landmarks
    landmarks = shape_to_np(shape)
    
   # Draw the landmark points with circles 
    for (x, y) in landmarks:
        annotated_image = cv2.circle(image, (x, y),2, (0, 127, 255), -1)

    return annotated_image, landmarks

The helper function below converts the shape object returned by the predictor function into a more convenient NumPy array. So the helper function below is being used by the landmark function we created above.

In [ ]:
def shape_to_np(shape): 
    
    # Create an array of shape (68, 2) for storing the landmark coordinates
    landmarks = np.zeros((68, 2), dtype="int")
    
    # Write the x,y coordinates of each landmark into the array
    for i in range(0, 68): 
        landmarks[i] = (shape.part(i).x, shape.part(i).y)
        
        
    return landmarks

Test detect_landmarks() function:


In [ ]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Landmark Detection', cv2.WINDOW_NORMAL) 

while(True):
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Get the landmarks for the face region in the frame
        landmark_image, landmarks = detect_landmarks(box_coords, frame)

    # Display the frame
    cv2.imshow('Landmark Detection',landmark_image)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything is done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 3: Jump Control mechanism


we will start implementing the jump control mechanism, what we need to do is to make the dino jump when the mouth is opened and stay still when it’s not.

In [ ]:
def is_mouth_open(landmarks, ar_threshold = 0.7): 
    
    
    # Calculate the euclidean distance labelled as A,B,C - FOR MOUTH OPEN / CLOSE
    A = hypot(landmarks[50][0] - landmarks[58][0], landmarks[50][1] - landmarks[58][1])
    B = hypot(landmarks[52][0] - landmarks[56][0], landmarks[52][1] - landmarks[56][1])
    C = hypot(landmarks[48][0] - landmarks[54][0], landmarks[48][1] - landmarks[54][1])
    
    # Calculate the mouth aspect ratio
    # The value of vertical distance A,B is averaged
    mouth_aspect_ratio = (A + B) / (2.0 * C)
    
    # Return True if the value is greater than the threshold
    if mouth_aspect_ratio > ar_threshold:
        return True, mouth_aspect_ratio
    else:
        return False, mouth_aspect_ratio

Testing the is_mouth_open() function:

In [ ]:
# get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Mouth Status', cv2.WINDOW_NORMAL)

while(True):
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Get the landmarks for the face region in the frame
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # Adjust the threshold and make sure it's working for you.
        mouth_status,_ = is_mouth_open(landmarks, ar_threshold = 0.6)
        
        # Display the mouth status
        cv2.putText(frame,'Is Mouth Open: {}'.format(mouth_status),
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)


    # Display the frame
    cv2.imshow('Mouth Status',frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 4: Crouch Control Mechanism



The Crouch control mechanism will utilize the euclidean distance between the top-left corner and bottom-right corner of the face bounding box. When the face is near the camera the distance will be greater, and when the face is close enough a key down event will be triggered causing the Dino to crouch.

In [ ]:
def face_proximity(box,image, proximity_threshold = 325):
    
    # Get the height and width of the face bounding box
    face_width =  box[2]-box[0]
    face_height = box[3]-box[1]
    
    # Draw rectangle to guide the user 
    # Calculate the angle of diagonal using face width, height 
    theta = np.arctan(face_height/face_width)
     
    # Use the angle to calculate height, width of the guide rectangle
    guide_height = np.sin(theta)*proximity_threshold
    guide_width  = np.cos(theta)*proximity_threshold
    
    # Calculate the mid-point of the guide rectangle/face bounding box
    mid_x,mid_y = (box[2]+box[0])/2 , (box[3]+box[1])/2
    
    #Calculate to coordinates of top-left and bottom-right corners
    guide_topleft = int(mid_x-(guide_width/2)), int(mid_y-(guide_height/2))
    guide_bottomright = int(mid_x +(guide_width/2)), int(mid_y + (guide_height/2))
    
    # Draw the guide rectangle
    cv2.rectangle(image, guide_topleft, guide_bottomright, (0, 255, 255), 2)
    
    # Calculate the diagonal distance of the bounding box
    diagonal = hypot(face_width, face_height)
    
    # Return True if distance greater than the threshold
    if diagonal > proximity_threshold:
        return True, diagonal
    else:
        return False, diagonal

Testing the face_proximity() function:


In [ ]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Face proximity', cv2.WINDOW_NORMAL)

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Check if face is closer than the defined threshold
        is_face_close,_ = face_proximity(box_coords, face_image, proximity_threshold = 325)
        
        # Display the mouth status
        cv2.putText(face_image,'Is Face Close: {}'.format(is_face_close),
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)

        
    # Display the frame
    cv2.imshow('Face proximity',face_image)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 5: Perform Calibration (Optional)
The threshold values I have selected for is_mouth_open() and face_proximity() should generalize well for most people but in case those values are not working for you, you can change it here.

So in this step, we’ll learn how to Calibrate the ideal threshold values. Now run the code below and open-close your mouth and find the threshold that works best to differentiate between the two states. Also, do the same thing for the face proximity threshold.

In [ ]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Calibration', cv2.WINDOW_NORMAL)

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Detect landmarks if the frame is found
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # Get the current mouth aspect ratio
        _,mouth_ar = is_mouth_open(landmarks)
    
        # Get the current face proximity
        _, proximity  = face_proximity(box_coords, face_image)

        # Calculate threshold values
        ar_threshold = mouth_ar*1.4
        proximity_threshold = proximity*1.3

        
        # Dsiplay the threshold values
        cv2.putText(frame, 'Aspect ratio threshold: {:.2f} '.format(ar_threshold), 
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)
        
        cv2.putText(frame,'Proximity threshold: {:.2f}'.format(proximity_threshold), 
                    (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)
     
    # Display the frame
    cv2.imshow('Calibration',frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything is done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 6: Keyboard Automation
Note: When running the following cells in Jupyter Notebook, make sure you don't use the Shift + Enter command to run the following code cells. You can use the Run code cell button in the Toolbar. Also if the keyboard buttons misbehave then you can also restart the kernel

The Dino cannot jump or crouch unless it is provided with input from the keyboard. We will use the output from the is_mouth_open() and face_proximity() functions to trigger keyboard keypress events.

For this, we will be using PyAutoGUI which is a simple API that lets a python script control the mouse and keyboard, it’s used to automate processes. Let’s have a look at how it works and how you can use it to trigger key events.

In [ ]:
# This will open a context menu
pyautogui.click(button='right')

To trigger key presses we can use press() function.



In [ ]:
# Press space bar. This will scroll down the page in some browsers
pyautogui.press('space')

To press multiple keys we can pass a list of strings to press() function.



In [ ]:
# This will move the focus to the next cell in the notebook
pyautogui.press(['shift','enter'])


When you use pyautogui.keyDown() instead of pyautogui.press(). Then the specified key is held down unless pyautogui.keyUp()event takes place helping us trigger a longer key press.

In [ ]:
# Hold down the shift key
pyautogui.keyDown('shift')
# Press enter while the shift key is down, this will run the next code cell
pyautogui.press('enter')

# Release the shift key
pyautogui.keyUp('shift')

In [ ]:
# This will run automatically after running the two code cells above
print('I ran')

## Step 7: Build The Final Application


Go to Chrome://Dino in your Chrome browser and run the code cell below.

Note: The image window screen will freeze when you trigger key buttons since at that moment the while loop will pause to press the key. So don't worry about that, after the program launches minimize the camera window and just go to chrome://dino/ and start playing using your face and mouth.

In [ ]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Lets Go! Play the Game', cv2.WINDOW_NORMAL)

# By default each key press is followed by a 0.1 second pause
pyautogui.PAUSE = 0.0

# The fail-safe triggers an exception in case mouse
# is moved to corner of the screen
#pyautogui.FAILSAFE = False

while(True):
    
     # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Detect landmarks if a face is found
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # Check if mouth is open
        is_open,_ = is_mouth_open(landmarks, ar_threshold)
        
        # If the mouth is open trigger space key Down event to jump
        if is_open:
            
            pyautogui.keyDown('space')
            mouth_status = 'Open'

        else:
            # Else the space key is Up
            pyautogui.keyUp('space')
            mouth_status = 'Closed'
        
        # Display the mouth status on the frame
        cv2.putText(frame,'Mouth: {}'.format(mouth_status),
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)
        
        # Check the proximity of the face
        is_closer,_  = face_proximity(box_coords, frame, proximity_threshold)
        
        # If face is closer press the down key
        if is_closer:
            pyautogui.keyDown('down')
            
        else:
            pyautogui.keyUp('down')
        
    # Display the frame
    cv2.imshow('Dino with OpenCV',frame)

    # Break the loop if ESC key pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()